# Topical Lectures April 2021
## Drone control, part 4a: test flight
Andreas Freise, Bas Swinkels 13.04.2021

In the 4th and last part of the project we provide two alternative notebooks, one which uses interactive plotting and one which uses non-interactive plotting. If you can we suggest that you use the one with interactive plots (4a) first.

In this notebook, we will try to fly our drone, using the controls we developped in the previous notebooks. To make it a bit more fun, we pose the following challenge: fly your drone through a set of predefined markers and report the time. We will compare the best times at the end of the project.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import module
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# interactive plots again
%matplotlib qt

# Don't forget to set the name string to your own name
MyName = 

First copy over Enter the functions for the drone feedback that you developed, i.e.
 - set_v()
 - set_v_nohover()
 - zphi2V()
(or similar functions you developed in the previous notebooks)



In [ ]:
# Enter the functions for the drone feedback here


Enter the functions to compute the feedback for the y,z and phi control loops below:

In [ ]:
# Functions for
# - vertical feedback 
# - tilt feedback 
# - horizontal feedback


## Racing your drone

In order to test our new control loops we are going to race our drone through a simple track. The goal is to reach a pre-defined set of targets as quickly as possible. In this notebook we will use an interactive plot and keyboard controls to do so.

Below we provide some code that allows keyboard control similar to notebook 1. However, this time the keys are used to change the set point of the y and z loops. As you will see, flying the drone has become much easier!

Add your code to compute the feddback signals at the indicated place.

In [ ]:
class Control():
    def __init__(self, _drone):
        self.drone = _drone
        self.plotter =  module.Plotter(self.drone)

        self.key_dy = 10
        self.key_dz = 10
        self.stop = False
        self.reset()

    def reset(self):
        self.V_left  = 0
        self.V_right = 0
        self.F = 0.5
        self.offset = 0
        self.y0 = 0 
        self.z0 = 0
        self.plotter.reset()

    def key_control(self):
        self.stop = False
        self.drone.stop = False
        self.plotter.fig.canvas.mpl_disconnect(self.plotter.fig.canvas.manager.key_press_handler_id)
        self.cid = self.plotter.fig.canvas.mpl_connect('key_press_event', self.on_key_simple)
        results = np.zeros(12)
        z_fb = 0
        y_fb = 0
        drone_str = ''

        while not self.stop:
            # update drone and store results
            results = self.drone.update()
            # setting local variables for better readability
            z = results[2]
            dz = results[5]
            y = results[1]
            dy = results[4]
            phi = results[3]
            dphi = results[6]

            # ----------------------------------------------------
            # Add your function calls to compute the feedback here.

            # ....

            # ----------------------------------------------------

            # Applying feedback
            self.V_left, self.V_right = zphi2V(z_fb, phi_fb)

            # ----------------------------------------------------
            # Change the below to your set_v function (which you
            # used to develop the feedback loops)
            set_v_nohover(self.drone,self.V_left, self.V_right)
            # ----------------------------------------------------

            # Checking if all targets have been reached, if so, stop!
            if results[11] == self.drone.num_targets:
                self.stop = True

            # String to print set point in interactive window
            drone_str = f"y0 = {self.y0}, z0 = {self.z0}"
            # Update the interatctive plot
            self.plotter.update_display(drone_str)
        self.plotter.fig.canvas.mpl_disconnect(self.cid)

        if self.drone.num_targets>0:
            if results[11] == self.drone.num_targets:
                print(f"All targets reached in: {results[0]:.2f} seconds.")
            else:
                print("Not all targets have been reached.")

    def on_key_simple(self, event):
        if event.key == 'escape':
            print('Stopping!')
            self.stop = True
        elif event.key == 'w':
            self.z0 += self.key_dz
        elif event.key == 's':
            self.z0 -= self.key_dz
        elif event.key == 'a':
            self.y0 -= self.key_dy
        elif event.key == 'd':
            self.y0 += self.key_dy


To test our controls we set a 'race' track as a set of targets. 

In [ ]:
drone = module.Drone(name=MyName, wind=True)
# Defining the track as an array of targets [y,z,radius].
# The targets have to be reached in the given order.
# The track below is the `offical' track, we want to see your best time for this exact track!
targets = np.array([
    [ 200,  300, 10],
    [ 200, -300, 10],
    [-200, -300, 10],
    [-200,  300, 10],
    [0,0,10]
])

# Setting the targets
drone.set_targets(targets)

control = Control(drone)

In [ ]:
control.key_control()

In [ ]:
control.plotter.close()